In [22]:
from dotenv import load_dotenv
load_dotenv()
import os
from openai import OpenAI
from pinecone import Pinecone, ServerlessSpec

In [17]:
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))
pc.create_index(
    name="rag", dimension=1536, metric="cosine", spec=ServerlessSpec(cloud="aws", region="us-east-1")
)

In [20]:
import json
data = json.load(open("reviews.json"))
data['reviews']

[{'professor': 'Dr. Emily Chen',
  'subject': 'Calculus II',
  'stars': 5,
  'review': 'Dr. Chen is an exceptional professor. She is extremely knowledgeable and passionate about the subject, and her explanations are clear and concise. The coursework is challenging but manageable, and she provides ample support during office hours. I highly recommend taking any course with Dr. Chen.'},
 {'professor': 'Professor John Smith',
  'subject': 'Principles of Microeconomics',
  'stars': 3,
  'review': 'Professor Smith is knowledgeable about the subject matter, but his lectures can be dry and difficult to follow at times. The grading seems a bit subjective, and I wish he provided more feedback on assignments.'},
 {'professor': 'Dr. Sarah Lee',
  'subject': 'Organic Chemistry I',
  'stars': 4,
  'review': 'Dr. Lee is passionate about the subject and really tries to help students understand the material. The lab component is well-organized, and she is always willing to answer questions during offi

In [23]:
processed_data = []
client=OpenAI()

for review in data['reviews']:
    response = client.embeddings.create(
        input = review['review'],
        model= "text-embedding-3-small",
    )
    embedding = response.data[0].embedding
    processed_data.append({
        "values":embedding,
        "id":review["professor"],
        "metadata":{
            "review":review["review"],
            "subject":review["subject"],
            "stars":review["stars"]
        }
    })

In [25]:
index =pc.Index('rag')
index.upsert(
    vectors=processed_data,
    namespace="ns1"
)

{'upserted_count': 21}